# Mouse snRNA Integrative Analysis
## Heart
### Data
- [Heart data table](https://github.com/erebboah/enc4_mouse/blob/master/snrna/ref/cortex_minimal_metadata.tsv)

### Aims
[integrate_parse_10x.R](https://github.com/erebboah/enc4_mouse/blob/master/snrna/scripts/integrate_parse_10x.R):
1. Read in pre-processed Parse and 10x data and merge counts matrices across experiments (within the same technology) for each tissue.
2. Filter nuclei by # genes, # UMIs, percent mitochondrial gene expression, and doublet score. See [detailed metadata](https://github.com/erebboah/enc4_mouse/blob/master/snrna/ref/enc4_mouse_snrna_metadata.tsv) for filter cutoffs. **Also filter 10x nuclei for those passing snATAC filters.**
3. Run SCT on the 3 objects to regress `percent.mt` and `nFeature_RNA`. Use  `method = "glmGamPoi"` to speed up this step, and save pre-integrated data in `seurat` folder.
4. Combine Parse standard, Parse deep, and 10x data by CCA integration. Use Parse standard as reference dataset because it contains all timepoints, while 10x data only contains 2 timepoints. 
5. Score nuclei by cell cycle using these [mouse cell cycle genes](https://github.com/erebboah/enc4_mouse/blob/master/snrna/ref/mouse_cellcycle_genes.rda) to aid in manual celltype annotation.

[predict_heart_celltypes.R](https://github.com/erebboah/enc4_mouse/blob/master/snrna/scripts/predict_cortex_celltypes.R): use 2 external datasets to predict celltypes.

- [Stressed mouse ventricles](https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-8810/files/) and associated [paper](https://www.ahajournals.org/doi/10.1161/CIRCULATIONAHA.119.045115)
- [Human heart cell atlas](https://www.heartcellatlas.org/) converted to mouse gene IDs.

**In this notebook**:
Manual celltype annotation by assigning each cluster to the celltype predicted for the majority of cells in the cluster, then adjusting the labels as we see fit. Find marker genes for `gen_celltype`, `celltypes`, and `subtypes` and save in `seurat/markers`.

### Results
- Stressed heart data did not include annotations, so I annotated them manually in [predict_heart_celltypes.R](https://github.com/erebboah/enc4_mouse/blob/master/snrna/scripts/predict_cortex_celltypes.R) using the marker genes in [Fig. 1C](https://www.ahajournals.org/cms/asset/5d52d0eb-9c13-4a10-81f4-58e18559d64d/circulationaha.119.045115.fig01.jpg)
- Main heart celltypes are CM (cardiomyocytes), CF (cardiac fibroblasts), and CE (cardiac endothelial)
- Split CM, CF, CE clusters by timepoint (early, middle, late) when appropriate
- Split other clusters by cycling
- Stressed heart data predicted most celltypes, but I used the adipocyte and atrial cardiomyocyte labelling information from the heart atlas data


In [114]:
library(Matrix)
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(viridis))
library(glmGamPoi)
library(RColorBrewer)
options(future.globals.maxSize = 10000 * 1024^2)
future.seed=TRUE


In [ ]:
setwd("../../enc4_mouse/snrna/")

In [115]:
setwd("/share/crsp/lab/seyedam/share/enc4_mouse/snrna/")

In [1]:
system("mkdir plots/heart/")
system("mkdir plots/heart/qc")
system("mkdir plots/heart/clustering")
system("mkdir plots/heart/annotation")
system("mkdir seurat/markers/heart")
system("mkdir ref/heart")

# Functions

In [3]:
get_orig_counts = function(file){
    metadata = metadata[metadata$file_accession == file,]
    counts = readMM(paste0("counts_10x/",file,"/matrix.mtx"))
    
    barcodes = read.delim(paste0("counts_10x/",file,"/barcodes.tsv"),header = F, col.names="barcode")
    features = read.delim(paste0("counts_10x/",file,"/genes.tsv"),header = F, col.names="gene_name") 
    colnames(counts) = barcodes$barcode
    rownames(counts) = features$gene_name
    out = counts

}

In [4]:
knee_df = function(mtx,expt_name){
    df = as.data.frame(rowSums(mtx))
    colnames(df) = c("nUMI")
    df <- tibble(total = df$nUMI,
               rank = row_number(dplyr::desc(total))) %>%
    distinct() %>%
    arrange(rank)
    df$experiment = expt_name
    out = df
}

# QC plots

## Knee plot

In [5]:
combined.sct = readRDS("seurat/heart_Parse_10x_integrated.rds")
cellbend_10x = subset(combined.sct,subset=technology =="10x")
orig_parse = subset(combined.sct,subset=technology =="Parse")
parse_standard = subset(orig_parse,subset=depth1 =="shallow")
parse_deep = subset(orig_parse,subset=depth1 =="deep")

In [6]:
metadata = read.delim("ref/enc4_mouse_snrna_metadata.tsv")
metadata = metadata[metadata$technology == "10x",]
metadata = metadata[metadata$tissue == "Heart",]

files = metadata$file_accession

orig_10x = get_orig_counts(files[1])

for (j in 2:length(files)){
    counts_adding = get_orig_counts(files[j])
    orig_10x = cbind(orig_10x, counts_adding)
}


In [7]:
cellbend_knee_plot = knee_df(cellbend_10x@assays$RNA@counts, "10x + Cellbender")
orig_knee_plot = knee_df(orig_10x, "10x")

parse_standard_knee_plot = knee_df(parse_standard@assays$RNA@counts, "Parse standard")
parse_deep_knee_plot = knee_df(parse_deep@assays$RNA@counts, "Parse deep")

pdf(file="plots/heart/qc/experiment_kneeplots.pdf",
    width = 10, height = 8)
ggplot(rbind(cellbend_knee_plot,orig_knee_plot,parse_standard_knee_plot,parse_deep_knee_plot), 
       aes(rank, total, group = experiment, color = experiment)) +
geom_path() + 
scale_y_log10() + scale_x_log10() + annotation_logticks() +
labs(y = "Total UMI count", x = "Barcode rank", title = "Mouse heart knee plot") + 
geom_hline(yintercept=500, linetype="dashed", color = "red", size=1)
dev.off()


Warning message:
"Transformation introduced infinite values in continuous y-axis"


png 
  2

In [8]:
pdf(file="plots/heart/qc/experiment_violinplots.pdf",
    width = 20, height = 5)
VlnPlot(combined.sct, features = c("nFeature_RNA"), ncol = 1, split.by = "depth2",
        pt.size = 0, group.by = "sample", cols = c("#811b74","#C08DBA","#00a1e0"))+ ggtitle("# genes per nucleus") +
stat_summary(fun.y = median, geom='point', size = 15, colour = "black", shape = 95) & theme(text = element_text(size = 20), 
                                                                              axis.text.x = element_text(size = 20), 
                                                                              axis.text.y = element_text(size = 20))
VlnPlot(combined.sct, features = c("nCount_RNA"), ncol = 1, split.by = "depth2",
        pt.size = 0, group.by = "sample", cols = c("#811b74","#C08DBA","#00a1e0")) + ggtitle("# UMIs per nucleus") +
stat_summary(fun.y = median, geom='point', size = 15, colour = "black", shape = 95)& theme(text = element_text(size = 20), 
                                                  axis.text.x = element_text(size = 20), 
                                                  axis.text.y = element_text(size = 20))
VlnPlot(combined.sct, features = c("percent.mt"), ncol = 1, split.by = "depth2",
        pt.size = 0, group.by = "sample", cols = c("#811b74","#C08DBA","#00a1e0")) & theme(text = element_text(size = 20), 
                                                  axis.text.x = element_text(size = 20), 
                                                  axis.text.y = element_text(size = 20)) 
dev.off()

The default behaviour of split.by has changed.
Separate violin plots are now plotted side-by-side.
To restore the old behaviour of a single split violin,
set split.plot = TRUE.
      
This message will be shown once per session.

Warning message:
"`fun.y` is deprecated. Use `fun` instead."
Warning message:
"`fun.y` is deprecated. Use `fun` instead."


png 
  2

## UMAP "Feature Plots" of QC metadata

In [9]:
png(file="plots/heart/qc/qc_featureplot.png",
    width = 1200, height = 500)

FeaturePlot(combined.sct, pt.size = 0.1,
            features =c("nFeature_RNA",
                        "nCount_RNA",
                        "percent.mt",
                        "percent.ribo",
                        "doublet_scores",
                        "G2M.Score"), ncol =3,order=T)  & scale_colour_gradientn(colours = viridis(11)) & 
                        NoAxes()& 
                        theme(text = element_text(size = 18))

dev.off()


Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.



png 
  2

# Check integration and clustering
Increased resolution to split a cluster that combined CF and CE.

In [ ]:
DefaultAssay(combined.sct) = "integrated"
combined.sct <- FindClusters(combined.sct,resolution=2,verbose = F)


In [223]:
nclusters = length(unique(combined.sct$seurat_clusters))
cluster_cols = colorRampPalette(brewer.pal(9,"Set1"))(nclusters)

In [224]:
pdf(file="plots/heart/clustering/UMAP_Parse_10x.pdf",
    width = 20, height = 8)
p1 <- DimPlot(combined.sct, reduction = "umap", group.by = "technology")
p2 <- DimPlot(combined.sct, reduction = "umap", label = TRUE, repel = TRUE, cols = cluster_cols)
p1 + p2

dev.off()

png 
  2

In [225]:
pdf(file="plots/heart/clustering/Parse_10x_experiment_distribution.pdf",
    width = 20, height = 6)
DimPlot(combined.sct, reduction = "umap", group.by = "seurat_clusters",split.by = "depth2", label = TRUE, label.size = 6, repel = TRUE, shuffle = T,cols = cluster_cols)

ggplot(combined.sct@meta.data, aes(x=seurat_clusters, fill=depth2)) + geom_bar(position = "fill") & 
theme(text = element_text(size = 20), axis.text.x = element_text(size = 20), axis.text.y = element_text(size = 20))

dev.off()


png 
  2

In [226]:
combined.sct$sample = factor(combined.sct$sample, levels=paste0("H_",rep(c("4","10","14","25","36","2m","18m"),each=4),rep(c("_M","_F"),each=2),c("_1","_2")))

pdf(file="plots/heart/clustering/UMAP_cluster_sample_barplot.pdf",
    width = 20, height = 10)
p1=DimPlot(combined.sct, reduction = "umap", group.by = "seurat_clusters", label = TRUE, label.size = 8, repel = TRUE, 
          cols = cluster_cols)
p2=ggplot(combined.sct@meta.data, aes(x=seurat_clusters, fill=sample)) + geom_bar(position = "fill") +
theme(text = element_text(size = 20), axis.text.x = element_text(size = 20), axis.text.y = element_text(size = 20)) & coord_flip()
gridExtra::grid.arrange(
  p1, p2,
  widths = c(2,1.6),
  layout_matrix = rbind(c(1, 2)))

dev.off()

png 
  2

In [227]:
pdf(file="plots/heart/clustering/age_sex_barplot.pdf",
    width = 18, height = 19)
p1=DimPlot(combined.sct, reduction = "umap", group.by = "timepoint", label = TRUE, label.size = 5, repel = TRUE)
p2 = ggplot(combined.sct@meta.data, aes(x=seurat_clusters, fill=timepoint)) + geom_bar(position = "fill") & 
theme(text = element_text(size = 20), axis.text.x = element_text(size = 20), axis.text.y = element_text(size = 20)) & coord_flip()

p3=DimPlot(combined.sct, reduction = "umap", group.by = "sex", label = TRUE, label.size = 5, repel = TRUE, shuffle = T)
p4 = ggplot(combined.sct@meta.data, aes(x=seurat_clusters, fill=sex)) + geom_bar(position = "fill") & 
theme(text = element_text(size = 20), axis.text.x = element_text(size = 20), axis.text.y = element_text(size = 20)) & coord_flip()
gridExtra::grid.arrange(
  p1, p2, p3, p4,
  widths = c(2,1),
  layout_matrix = rbind(c(1, 2),
                        c(3, 4)))

dev.off()

png 
  2

# Plotting: check predicted celltypes

In [228]:
pdf(file="plots/heart/annotation/UMAP_predictions.pdf",
    width = 15, height = 12)
nclusters = length(unique(combined.sct$predictions))
DimPlot(combined.sct, reduction = "umap", group.by = "predictions",
        label = TRUE, label.size = 6, repel = TRUE,cols = colorRampPalette(brewer.pal(9,"Set1"))(nclusters)) + NoLegend()

nclusters = length(unique(combined.sct$atlas_predictions))
DimPlot(combined.sct, reduction = "umap", group.by = "atlas_predictions",
        label = TRUE, label.size = 6, repel = TRUE,cols = colorRampPalette(brewer.pal(9,"Set1"))(nclusters)) + NoLegend()

dev.off()

png 
  2

# Rename clusters based on maximum predicted celltype

In [229]:
Idents(combined.sct) ="seurat_clusters"
mat = as.matrix(table(Idents(combined.sct), combined.sct$atlas_predictions))
ct = colnames(mat)[max.col(mat)]
names(ct) = 0:(length(ct)-1)

# basically add the cluster info to the maximum predicted celltype
for (i in 1:length(unique(Idents(combined.sct))))
{
    search = paste0("\\<",names(ct)[i],"\\>")
    replace = paste0(ct[i],".",names(ct)[i])
    Idents(combined.sct) = gsub(search,replace,Idents(combined.sct))
}

combined.sct[["atlas_celltypes"]] <- Idents(combined.sct)

In [230]:
Idents(combined.sct) = "seurat_clusters"
mat = as.matrix(table(Idents(combined.sct), combined.sct$predictions))
ct = colnames(mat)[max.col(mat)]
names(ct) = 0:(length(ct)-1)

# basically add the cluster info to the maximum predicted celltype
for (i in 1:length(unique(Idents(combined.sct))))
{
    search = paste0("\\<",names(ct)[i],"\\>")
    replace = paste0(ct[i],".",names(ct)[i])
    Idents(combined.sct) = gsub(search,replace,Idents(combined.sct))
}

combined.sct[["stressed_celltypes"]] <- Idents(combined.sct)

In [231]:
pdf(file="plots/heart/annotation/UMAP_maximum_predictions.pdf",
    width = 15, height = 12)
nclusters = length(unique(combined.sct$stressed_celltypes))
DimPlot(combined.sct, reduction = "umap", group.by = "stressed_celltypes",
        label = TRUE, label.size = 6, repel = TRUE,cols = colorRampPalette(brewer.pal(9,"Set1"))(nclusters))  & NoLegend()

nclusters = length(unique(combined.sct$atlas_celltypes))
DimPlot(combined.sct, reduction = "umap", group.by = "atlas_celltypes",
        label = TRUE, label.size = 6, repel = TRUE,cols = colorRampPalette(brewer.pal(9,"Set1"))(nclusters))  & NoLegend()


dev.off()

png 
  2

# Manual celltype annotation

## Fix clusters

In [244]:
combined.sct$subtypes = combined.sct$stressed_celltypes # this slot is closer to what my final celltypes will be
combined.sct$subtypes = gsub("[+]","",combined.sct$subtypes )

combined.sct$subtypes = gsub("\\<Smooth_muscle.31\\>","Smooth_muscle1.31",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Fibroblasts.38\\>","Smooth_muscle2.38",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<DC-like_cells.47\\>","Pericytes_cycling.47",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Smooth_muscle.16\\>","Pericytes.16",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Smooth_muscle.13\\>","Pericytes.13",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Fibroblasts.41\\>","CF_cycling.41",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Fibroblasts.3\\>","CF_late.3",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<DC-like_cells.35\\>","CE_cycling.35",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<DC-like_cells.36\\>","CM_cycling.36",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Fibroblasts.17\\>","CM_atrial.17",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Fibroblasts.48\\>","CM_atrial.48",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Fibroblasts.23\\>","CM_int.43",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Fibroblasts.22\\>","CM_int.43",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Cardiomyocytes.27\\>","CM_int.27",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Cardiomyocytes.40\\>","CM_late.40",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Cardiomyocytes.24\\>","CM_int.24",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Cardiomyocytes.1\\>","CM_int.1",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Cardiomyocytes.0\\>","CM_int.0",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Cardiomyocytes.6\\>","CM_int.6",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Cardiomyocytes.2\\>","CM_int.2",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Cardiomyocytes.4\\>","CM_late.4",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Cardiomyocytes.21\\>","CM_early.21",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Endothelial.9\\>","CE_late.9",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Endothelial.18\\>","CE_late.18",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Fibroblasts.Wif1.45\\>","Adipocytes.45",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<DC-like_cells.44\\>","Macrophages_cycling.44",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<T_cells.28\\>","Lymphoid.28",combined.sct$subtypes) 



In [245]:
# get rid of cluster #
combined.sct$subtypes = do.call("rbind", strsplit(as.character(combined.sct$subtypes), "[.]"))[,1]
combined.sct$subtypes = gsub("\\<Cardiomyocytes\\>","CM",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Fibroblasts\\>","CF",combined.sct$subtypes) 
combined.sct$subtypes = gsub("\\<Endothelial\\>","CE",combined.sct$subtypes) 


## Check final clusters with marker gene dotplot

In [247]:
DefaultAssay(combined.sct) = "SCT"
genes = c("Ryr2","Ttn","Myh6","Gata4",
          "Flt1","Pecam1","Ptprb",
          "Lyve1","Prox1",
          "Mgp","Dcn","Col1a1",
          "Pdgfrb","Abcc9",
          "Myh11","Mylk","Eln","Myl6",
          "Nrg1","Heg1", # endocard
          "Slit3","Abi1",
          "F13a1",
          "Ikzf1","Ikzf3","Bcl11a",
          "Themis","Il7r",
          "Adipoq","Pnpla2", # adipocytes
          "Sox10","Cadm2","Postn",
          "Mki67","Top2a")# cycling

In [248]:
pdf(file="plots/heart/annotation/subtypes_marker_dotplot.pdf",
    width = 20, height = 10)
options(repr.plot.width = 20,repr.plot.height = 10)
DefaultAssay(combined.sct) = "SCT"
Idents(combined.sct) = "subtypes"
Idents(combined.sct) = factor(Idents(combined.sct), 
                              levels = c("CM_early","CM_int",
                                         "CM_late",
                                         "CM_cycling","CM_atrial",
                                         "CE","CE_late",
                                         "CE_cycling","Lymphatic_ECs","CF",
                                         "CF_late","CF_cycling",
                                         "Pericytes","Pericytes_cycling",
                                         "Smooth_muscle1","Smooth_muscle2",
                                         "Endocardial","Epicardial",
                                         "Macrophages","Macrophages_cycling",
                                         "Lymphoid",
                                         "Adipocytes",
                                         "Schwann_cells")) 
DotPlot(combined.sct, features = genes)+ theme(axis.text.x = element_text(angle = 45, hjust = 1)) 
dev.off()

png 
  2

# Add celltypes and gen_celltypes metadata
Based on the subtypes annotation, we can group the cells into broader categories.

In [249]:
combined.sct$celltypes = combined.sct$subtypes
combined.sct$celltypes = gsub("\\<CF_cycling\\>","CF",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<CF_late\\>","CF",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<CM_atrial\\>","CM",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<CM_early\\>","CM",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<CM_late\\>","CM",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<CM_int\\>","CM",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<CM_ribo\\>","CM",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<CM_cycling\\>","CM",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<CE_cycling\\>","CE",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<CE_late\\>","CE",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<Macrophages_cycling\\>","Macrophages",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<Pericytes_cycling\\>","Pericytes",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<Smooth_muscle2\\>","Smooth_muscle",combined.sct$celltypes)
combined.sct$celltypes = gsub("\\<Smooth_muscle1\\>","Smooth_muscle",combined.sct$celltypes)

In [250]:
combined.sct$gen_celltype = combined.sct$celltypes

combined.sct$gen_celltype = gsub("\\<Macrophages\\>","Myeloid",combined.sct$gen_celltype)
combined.sct$gen_celltype = gsub("\\<Epicardial\\>","Stromal",combined.sct$gen_celltype)
combined.sct$gen_celltype = gsub("\\<Endocardial\\>","Stromal",combined.sct$gen_celltype)
combined.sct$gen_celltype = gsub("\\<Lymphatic_ECs\\>","Endothelial",combined.sct$gen_celltype)
combined.sct$gen_celltype = gsub("\\<CE\\>","Endothelial",combined.sct$gen_celltype)
combined.sct$gen_celltype = gsub("\\<CF\\>","Cardiac_fibroblast",combined.sct$gen_celltype)
combined.sct$gen_celltype = gsub("\\<CM\\>","Cardiomyocyte",combined.sct$gen_celltype)

# Plotting the 3 levels of annotations

In [251]:
color_ref = read.delim("ref/enc4_mouse_snrna_celltypes_c2c12.csv",sep=",",col.names = c("tissue","gen_celltype","celltypes",
                                                                              "subtypes","gen_celltype_color",
                                                                              "celltype_color","subtype_color"))
gen_celltype_colors = unique(color_ref[color_ref$tissue == "Heart",c("gen_celltype","gen_celltype_color")])
rownames(gen_celltype_colors) = gen_celltype_colors$gen_celltype
gen_celltype_colors = gen_celltype_colors[sort(unique(combined.sct$gen_celltype)),]

pdf(file="plots/heart/annotation/UMAP_final_gen_celltype.pdf",
   width = 15, height = 10)

DimPlot(combined.sct, reduction = "umap", 
        group.by = "gen_celltype", 
        label = TRUE, label.size = 8, repel = TRUE,
       cols = gen_celltype_colors$gen_celltype_color)

dev.off()

png 
  2

In [254]:
color_ref = read.delim("ref/enc4_mouse_snrna_celltypes_c2c12.csv",sep=",",col.names = c("tissue","gen_celltype","celltypes",
                                                                              "subtypes","gen_celltype_color",
                                                                              "celltype_color","subtype_color"))
celltype_colors = unique(color_ref[color_ref$tissue == "Heart",c("celltypes","celltype_color")])
rownames(celltype_colors) = celltype_colors$celltypes
celltype_colors = celltype_colors[sort(unique(combined.sct$celltypes)),]

pdf(file="plots/heart/annotation/UMAP_final_celltypes.pdf",
    width = 15, height = 10)

DimPlot(combined.sct, reduction = "umap", 
        group.by = "celltypes", 
        label = TRUE, label.size = 8, repel = TRUE,
       cols = celltype_colors$celltype_color)

dev.off()

png 
  2

In [255]:
color_ref = read.delim("ref/enc4_mouse_snrna_celltypes_c2c12.csv",sep=",",col.names = c("tissue","gen_celltype","celltypes",
                                                                              "subtypes","gen_celltype_color",
                                                                              "celltype_color","subtype_color"))
subtype_colors = unique(color_ref[color_ref$tissue == "Heart",c("subtypes","subtype_color")])
rownames(subtype_colors) = subtype_colors$subtypes
subtype_colors = subtype_colors[sort(unique(combined.sct$subtypes)),]

pdf(file="plots/heart/annotation/UMAP_final_subtypes.pdf",
    width = 15, height = 10)

DimPlot(combined.sct, reduction = "umap", 
        group.by = "subtypes", 
        label = TRUE, label.size = 6, repel = TRUE,
       cols = subtype_colors$subtype_color)

dev.off()

png 
  2

## Proportion plot of celltypes over timepoint

In [256]:
combined.sct_parse = subset(combined.sct,subset= technology == "Parse")

samples = sort(unique(combined.sct_parse$timepoint))
dflist = list()
for (i in 1:length(unique(combined.sct_parse$timepoint))){
  tp=combined.sct_parse@meta.data[combined.sct_parse@meta.data$timepoint == samples[i],]
  #tp=tp[complete.cases(tp),]
  tp_df=as.data.frame(table(tp$celltypes))
  tp_df$percentage=tp_df$Freq/nrow(tp)
  tp_df$timepoint=rep(i,nrow(tp_df))
  dflist[[i]]=tp_df
}
df = do.call(rbind, dflist)
df <- df[order(df$timepoint),]
colnames(df)= c("celltypes","Freq","percentage","timepoint")



In [257]:
pdf(file="plots/heart/annotation/timepoint_celltypes_proportions.pdf",
    width = 15, height = 10)

ggplot(df, aes(x=timepoint, y=percentage, fill=celltypes)) + 
  geom_area()  +
  scale_fill_manual(values= celltype_colors$celltype_color) + 
  scale_x_continuous(breaks = c(1,2,3,4,5,6,7),labels= c("PND_04","PND_10","PND_14",
                                                         "PND_25","PND_36","PNM_02","PNM_18-20"))+
  scale_y_continuous(breaks = c(0,0.1,0.2,0.3,0.4,0.5,
                                0.6,0.7,0.8,0.9,1.0),labels= c("0%","10%","20%","30%","40%","50%","60%","70%","80%","90%","100%")) + 
theme_minimal()+theme(text = element_text(size = 30)) + 
theme(axis.text.x = element_text(size = 30))  + 
theme(axis.text.y = element_text(size = 30))   + 
theme(axis.text.x = element_text(angle = 45, hjust = 1))
  
dev.off()

png 
  2

# Save 

In [260]:
saveRDS(combined.sct,file="seurat/heart_Parse_10x_integrated.rds")

# Save embedding coordinates file

In [259]:
cell_id = colnames(combined.sct)
coords = cbind(cell_id,combined.sct@reductions$umap@cell.embeddings,
               combined.sct@reductions$pca@cell.embeddings)
fn = paste0("ref/heart/heart_integrated_analysis_embedding_coordinates_noheader.tsv")
write.table(coords,file=fn,
            sep="\t",
            quote=F,
            row.names = F)
out = paste0("ref/heart/heart_integrated_analysis_embedding_coordinates.tsv")
system(paste0("cat ref/embedding_coordinates_header ", fn, "> ", out))
system(paste0("rm ", fn))

# Get marker genes

In [ ]:
combined.sct  = readRDS("seurat/heart_Parse_10x_integrated.rds")

In [ ]:
gene_id_to_name = read.csv("ref/gene_id_to_name.csv",row.names = 1)
gene_id_to_name$seurat_gene_name = rownames(combined.sct@assays$RNA)

In [ ]:
format_marker_genes = function(seurat_markers){
    markers$seurat_gene_name = markers$gene
    markers = left_join(markers, gene_id_to_name)
    markers = markers[,c("gene_id","gene_name","p_val",
                     "avg_log2FC","pct.1","pct.2",
                     "p_val_adj","cluster")]
    out = markers
}

In [ ]:
DefaultAssay(combined.sct)= "RNA"
Idents(combined.sct) = "gen_celltype"
markers <- FindAllMarkers(combined.sct,
                             only.pos = TRUE, 
                             min.pct = 0.1, 
                             logfc.threshold = 0.25, 
                             verbose = T)
markers_formatted = format_marker_genes(markers)

write.table(markers_formatted,file=paste0("seurat/markers/heart/heart_gen_celltype_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25.tsv"),
            sep="\t",
            quote=F,
            row.names = F)


In [ ]:
for (celltype in unique(markers_formatted$cluster)){
    keep = c("gene_id","gene_name","p_val",
             "avg_log2FC","pct.1","pct.2",
             "p_val_adj")
    markers_celltype = markers_formatted[markers_formatted$cluster == celltype,]
    markers_celltype$is_enriched = 1
    markers_celltype = markers_celltype[,c("gene_id","gene_name","is_enriched","p_val","avg_log2FC","pct.1","pct.2","p_val_adj")]    
    fn = paste0("seurat/markers/heart/heart_gen_celltype_",
                                             celltype,"_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25_noheader.tsv")
    write.table(markers_celltype,file=fn,
            sep="\t",
            quote=F,
            row.names = F)
    
    out = paste0("seurat/markers/heart/heart_gen_celltype_",
                                             celltype,"_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25.tsv")
    system(paste0("cat ref/marker_gene_header ", fn, "> ", out))
    system(paste0("rm ", fn))
}

In [ ]:
DefaultAssay(combined.sct)= "RNA"
Idents(combined.sct) = "celltypes"
markers <- FindAllMarkers(combined.sct,
                             only.pos = TRUE, 
                             min.pct = 0.1, 
                             logfc.threshold = 0.25, 
                             verbose = T)
markers_formatted = format_marker_genes(markers)

write.table(markers_formatted,file=paste0("seurat/markers/heart/heart_celltypes_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25.tsv"),
            sep="\t",
            quote=F,
            row.names = F)


In [ ]:
for (celltype in unique(markers_formatted$cluster)){
    keep = c("gene_id","gene_name","p_val",
             "avg_log2FC","pct.1","pct.2",
             "p_val_adj")
    markers_celltype = markers_formatted[markers_formatted$cluster == celltype,]
    markers_celltype$is_enriched = 1
    markers_celltype = markers_celltype[,c("gene_id","gene_name","is_enriched","p_val","avg_log2FC","pct.1","pct.2","p_val_adj")]    
    fn = paste0("seurat/markers/heart/heart_celltypes_",
                                             celltype,"_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25_noheader.tsv")
    write.table(markers_celltype,file=fn,
            sep="\t",
            quote=F,
            row.names = F)
    
    out = paste0("seurat/markers/heart/heart_celltypes_",
                                             celltype,"_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25.tsv")
    system(paste0("cat ref/marker_gene_header ", fn, "> ", out))
    system(paste0("rm ", fn))
}

In [ ]:
DefaultAssay(combined.sct)= "RNA"
Idents(combined.sct) = "subtypes"
markers <- FindAllMarkers(combined.sct,
                             only.pos = TRUE, 
                             min.pct = 0.1, 
                             logfc.threshold = 0.25, 
                             verbose = T)
markers_formatted = format_marker_genes(markers)

write.table(markers_formatted,file=paste0("seurat/markers/heart/heart_subtypes_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25.tsv"),
            sep="\t",
            quote=F,
            row.names = F)


In [ ]:
for (celltype in unique(markers_formatted$cluster)){
    keep = c("gene_id","gene_name","p_val",
             "avg_log2FC","pct.1","pct.2",
             "p_val_adj")
    markers_celltype = markers_formatted[markers_formatted$cluster == celltype,]
    markers_celltype$is_enriched = 1
    markers_celltype = markers_celltype[,c("gene_id","gene_name","is_enriched","p_val","avg_log2FC","pct.1","pct.2","p_val_adj")]    
    fn = paste0("seurat/markers/heart/heart_subtypes_",
                                             celltype,"_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25_noheader.tsv")
    write.table(markers_celltype,file=fn,
            sep="\t",
            quote=F,
            row.names = F)
    
    out = paste0("seurat/markers/heart/heart_subtypes_",
                                             celltype,"_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25.tsv")
    system(paste0("cat ref/marker_gene_header ", fn, "> ", out))
    system(paste0("rm ", fn))
}

# Save cell type labels file

In [ ]:
get_membership_scores = function(seurat_markers,slot){
    celltype = unique(seurat_markers$cluster)
    celltype_metadata_scored = list()
    
    # for each celltyp
    for (i in 1:length(celltype)){ 
        celltype_markers = seurat_markers[seurat_markers$cluster == celltype[i],]
        combined.sct[["membership_score"]] = PercentageFeatureSet(combined.sct,
                                                              pattern = NULL,
                                                              features = celltype_markers$gene_name,
                                                              assay = "RNA")
        celltype_metadata = combined.sct@meta.data
        celltype_metadata = celltype_metadata[,c("cellID",slot,"membership_score")]
        celltype_metadata_scored[[i]] = celltype_metadata[celltype_metadata[,2] == celltype[i], ]

    }
    celltype_metadata_scored = do.call(rbind.data.frame, celltype_metadata_scored)
}


# gen_celltype
markers = read.delim("seurat/markers/heart/heart_gen_celltype_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25.tsv")
gen_celltype_membership = get_membership_scores(markers,"gen_celltype")
colnames(gen_celltype_membership) = c("cell_id","general_cell_type_name","general_cell_type_membership_score")

# celltypes
markers = read.delim("seurat/markers/heart/heart_celltypes_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25.tsv")
celltype_membership = get_membership_scores(markers,"celltypes")
colnames(celltype_membership) = c("cell_id","cell_type_name","cell_type_membership_score")

# subtypes
markers = read.delim("seurat/markers/heart/heart_subtypes_marker_genes_only.pos_min.pct0.1_logfc.threshold0.25.tsv")
subtype_membership = get_membership_scores(markers,"subtypes")
colnames(subtype_membership) = c("cell_id","sub_cell_type_name","sub_type_membership_score")

fn = paste0("ref/heart_integrated_analysis_cell_type_labels_noheader.tsv")
write.table(membership_metadata,file=fn,
            sep="\t",
            quote=F,
            row.names = F)
out = paste0("ref/heart/heart_integrated_analysis_cell_type_labels.tsv")
system(paste0("cat ref/celltype_labels_header ", fn, "> ", out))
system(paste0("rm ", fn))


## Get unfiltered RNA information

In [272]:
get_orig_10x_counts = function(file){
    metadata = metadata[metadata$file_accession == file,]
    counts = readMM(paste0("counts_10x/",file,"/matrix.mtx"))
    
    barcodes = read.delim(paste0("counts_10x/",file,"/barcodes.tsv"),header = F, col.names="barcode")
    features = read.delim(paste0("counts_10x/",file,"/genes.tsv"),header = F, col.names="gene_name") 
    colnames(counts) = barcodes$barcode
    rownames(counts) = features$gene_name
    out = counts

}

metadata = read.delim("ref/enc4_mouse_snrna_metadata.tsv")
metadata = metadata[metadata$technology == "10x",]
metadata = metadata[metadata$tissue == "Heart",]

files = unique(metadata$file_accession)

orig_10x = get_orig_10x_counts(files[1])

for (j in 2:length(files)){
    counts_adding = get_orig_10x_counts(files[j])
    orig_10x = cbind(orig_10x, counts_adding)
}


In [273]:
get_orig_parse_counts = function(file){
    metadata = metadata[metadata$file_accession == file,]
    counts = readMM(paste0("scrublet/",file,"_matrix.mtx"))
    
    barcodes = read.delim(paste0("scrublet/",file,"_barcodes.tsv"),header = F, col.names="barcode")
    features = read.delim(paste0("scrublet/",file,"_genes.tsv"),header = F, col.names="gene_name") 
    colnames(counts) = barcodes$barcode
    rownames(counts) = features$gene_name
    out = counts

}

metadata = read.delim("ref/enc4_mouse_snrna_metadata.tsv")
metadata = metadata[metadata$technology == "Parse",]
metadata = metadata[metadata$tissue == "Heart",]

files = unique(metadata$experiment_batch)

orig_parse = get_orig_parse_counts(files[1])

for (j in 2:length(files)){
    counts_adding = get_orig_parse_counts(files[j])
    orig_parse = cbind(orig_parse, counts_adding)
}

## Join unfiltered RNA and ATAC metadata

In [274]:
metadata = read.delim("../enc4_mouse_metadata.tsv")
orig_rna = cbind(orig_10x, orig_parse)
obj = CreateSeuratObject(counts = orig_rna, min.cells = 0, min.features = 0)
obj[["percent.mt"]] = PercentageFeatureSet(obj, pattern = "^mt-")
obj[["percent.ribo"]] <- PercentageFeatureSet(obj, pattern = "^Rp[sl][[:digit:]]|^Rplp[[:digit:]]|^Rpsa")
obj$cellID = colnames(obj)
obj$rna_library_accession = do.call("rbind", strsplit(as.character(obj$cellID), "[.]"))[,2]


rna_metadata = left_join(obj@meta.data[,c("cellID","rna_library_accession","nCount_RNA","percent.mt","percent.ribo")],
                         metadata)


Warning message:
"Non-unique features (rownames) present in the input matrix, making unique"
Joining, by = "rna_library_accession"


In [275]:
atac_metadata = read.csv("../snatac/ref/atac_unfiltered_metadata_all_tissues.csv")
atac_metadata = atac_metadata[,c("cellID","Sample","nFrags","TSSEnrichment","original_atac_bc")]
atac_metadata$rna_library_accession = atac_metadata$Sample
atac_metadata$Sample = NULL
atac_metadata = left_join(atac_metadata, metadata)
atac_metadata = atac_metadata[atac_metadata$tissue == "Heart",]


Joining, by = "rna_library_accession"


In [276]:
combined_metadata = full_join(rna_metadata,atac_metadata)
combined_metadata$rna_bc = do.call("rbind", strsplit(as.character(combined_metadata$cellID), "[.]"))[,1]
combined_metadata$passed_filtering = as.numeric(combined_metadata$cellID %in% combined.sct$cellID)


Joining, by = c("cellID", "rna_library_accession", "technology", "species",
"tissue", "sex", "timepoint", "rep", "sample", "depth1", "depth2",
"experiment", "experiment_batch", "integration_batch", "run_number",
"lower_nCount_RNA", "upper_nCount_RNA", "lower_nFeature_RNA",
"upper_doublet_scores", "upper_percent.mt", "rna_experiment_accession",
"rna_file_accession", "multiome_experiment_accession",
"atac_experiment_accession", "atac_library_accession", "atac_file_accession",
"minTSS", "minFrags")


# Save cell metadata file

In [277]:
combined_metadata = combined_metadata[,c("cellID",
                                         "rna_library_accession","rna_bc",
                                         "atac_library_accession","original_atac_bc",
                                         "nCount_RNA","nFrags",
                                         "percent.mt","percent.ribo","TSSEnrichment","passed_filtering",
                                        "rna_experiment_accession","atac_experiment_accession",
                                         "rna_file_accession","atac_file_accession","multiome_experiment_accession",
                                         "sample","technology","tissue","sex","timepoint","rep")]
colnames(combined_metadata) = c("cell_id","rna_dataset","rna_barcode","atac_dataset","atac_barcode",
                                "rna_umi_count","atac_fragment_count","rna_frac_mito","rna_frac_ribo","atac_tss_enrichment","passed_filtering",
                                        "rna_experiment_accession","atac_experiment_accession",
                                         "rna_file_accession","atac_file_accession","multiome_experiment_accession",
                                         "sample","technology","tissue","sex","timepoint","rep")


In [278]:
fn = paste0("ref/heart/heart_integrated_analysis_cell_metadata_noheader.tsv")
write.table(combined_metadata,file=fn,
            sep="\t",
            quote=F,
            row.names = F)
out = paste0("ref/heart/heart_integrated_analysis_cell_metadata.tsv")
system(paste0("cat ref/cell_metadata_header ", fn, "> ", out))
system(paste0("rm ", fn))

# Cerebro

In [ ]:
combined.sct = readRDS("seurat/heart_Parse_10x_integrated.rds")


In [ ]:
# also add markers for my annotated cell types
combined.sct <- cerebroApp::getMarkerGenes(combined.sct, organism = 'mm',groups = c('subtypes'), 
                                  min_pct = 0.1,thresh_logFC = 0.25, assay = "RNA")

combined.sct <- cerebroApp::getEnrichedPathways(combined.sct, adj_p_cutoff = 0.01, max_terms = 100)


DefaultAssay(combined.sct) = "integrated"

combined.sct <- RunUMAP(
  combined.sct,
  reduction.name = 'UMAP_3D',
  reduction.key = 'UMAP3D_',
  dims = 1:30,
  n.components = 3,
  seed.use = 100
)


library(cerebroApp)
exportFromSeurat(
  object = combined.sct,
  assay = "SCT",
  slot = "data",
  file = "seurat/heart_Parse_10x_integrated.crb",
  experiment_name = "Mouse heart",
  organism = "mm",
  groups = c('doublet_scores','doublets','library_accession','technology','species',
             'tissue','sex','timepoint','rep','sample','depth1','depth2','experiment',
             'experiment_batch','integration_batch','run_number','experiment_accession',
             'file_accession','atlas_predictions','S.Score','G2M.Score',
             'Phase','celltypes','gen_celltype','atlas_celltypes','subtypes','seurat_clusters','predicted.id'),
  nUMI = 'nCount_RNA',
  nGene = 'nFeature_RNA',
  use_delayed_array = FALSE,
  verbose = TRUE
)